In [ ]:
!pip install pytorch-tabnet

In [ ]:
import pandas as pd
train = pd.read_csv('train_290_without_low1.csv')
test = pd.read_csv('test_dropped_290_without_nans.csv')
print(train.shape, test.shape)
import torch

(2828, 294) (939, 293)


In [ ]:
y_train=train['Culture']#pd.read_csv('y_train.csv')['Culture']
train.drop('Culture',axis=1,inplace=True)
y_train.shape

(2828,)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetClassifier

Device used : cuda


In [ ]:
folds = 5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 7)

In [ ]:
from sklearn.metrics import f1_score
predss=pd.DataFrame()
i=0
for train_index, test_index in skf.split(train, y_train):
  x_tt, x_tv = train.values[train_index,:], train.values[test_index,:]
  y_tt, y_tv = y_train.values[train_index], y_train.values[test_index]
  clf = TabNetClassifier(
    n_d=64, n_a=64, n_steps=7,
    gamma=1.5, n_independent=2, n_shared=4,
    lambda_sparse=1e-4, momentum=0.3, clip_value=2.,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95,
                     "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15)
  clf.fit(
      X_train = x_tt,
        y_train = y_tt,
        eval_set = [(x_tv, y_tv)],
        eval_name = ["val"],
        max_epochs = 1200,
        patience = 200,
        batch_size = 1024, 
        virtual_batch_size = 32,
        num_workers = 2)
  preds = clf.predict(x_tv)
  print(f1_score(preds,y_tv,average='weighted'))
  preds=clf.predict(test.values)
  predss['pred'+str(i)]=pd.Series(preds)
  i+=1

Выходные данные были обрезаны до нескольких последних строк (5000).
epoch 326| loss: 0.47516 | val_accuracy: 0.81449 |  0:06:10s
epoch 327| loss: 0.45527 | val_accuracy: 0.80389 |  0:06:11s
epoch 328| loss: 0.48513 | val_accuracy: 0.81625 |  0:06:12s
epoch 329| loss: 0.46583 | val_accuracy: 0.81979 |  0:06:13s
epoch 330| loss: 0.48308 | val_accuracy: 0.80212 |  0:06:15s
epoch 331| loss: 0.47834 | val_accuracy: 0.79329 |  0:06:16s
epoch 332| loss: 0.46625 | val_accuracy: 0.80035 |  0:06:17s
epoch 333| loss: 0.47588 | val_accuracy: 0.81449 |  0:06:18s
epoch 334| loss: 0.45397 | val_accuracy: 0.81979 |  0:06:19s
epoch 335| loss: 0.44015 | val_accuracy: 0.81095 |  0:06:20s
epoch 336| loss: 0.48415 | val_accuracy: 0.81802 |  0:06:21s
epoch 337| loss: 0.46655 | val_accuracy: 0.80565 |  0:06:22s
epoch 338| loss: 0.47173 | val_accuracy: 0.81272 |  0:06:23s
epoch 339| loss: 0.46332 | val_accuracy: 0.83216 |  0:06:24s
epoch 340| loss: 0.47288 | val_accuracy: 0.83216 |  0:06:26s
epoch 341| loss: 

In [ ]:
predss

,pred0,pred1,pred2,pred3,pred4
0,4,4,4,4,4
1,12,12,12,12,12
2,16,21,21,21,16
3,12,7,12,16,7
4,4,4,4,4,4
...,...,...,...,...,...
934,7,7,7,21,7
935,7,21,7,7,21
936,7,7,2,21,7
937,12,12,12,12,12


In [ ]:
final_preds=[]
for i in range(len(predss)):
  final_preds.append(predss.loc[i].value_counts().index.tolist()[0])
final_preds

[4,
 12,
 21,
 7,
 4,
 4,
 21,
 5,
 21,
 4,
 4,
 7,
 4,
 5,
 9,
 9,
 4,
 7,
 5,
 5,
 4,
 3,
 7,
 7,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 2,
 5,
 7,
 7,
 10,
 4,
 7,
 23,
 5,
 14,
 3,
 7,
 4,
 10,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 2,
 12,
 21,
 4,
 16,
 7,
 4,
 10,
 7,
 7,
 3,
 5,
 10,
 4,
 7,
 4,
 7,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 9,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 11,
 4,
 4,
 4,
 4,
 4,
 7,
 4,
 4,
 4,
 9,
 7,
 4,
 4,
 4,
 4,
 1,
 8,
 4,
 4,
 4,
 5,
 4,
 4,
 10,
 4,
 21,
 9,
 4,
 4,
 21,
 4,
 4,
 4,
 4,
 3,
 4,
 9,
 4,
 5,
 12,
 11,
 7,
 4,
 21,
 4,
 4,
 4,
 9,
 4,
 9,
 4,
 7,
 4,
 21,
 7,
 4,
 4,
 21,
 4,
 14,
 4,
 4,
 4,
 4,
 5,
 9,
 4,
 10,
 12,
 4,
 8,
 4,
 4,
 5,
 4,
 4,
 4,
 5,
 2,
 4,
 4,
 7,
 5,
 10,
 7,
 4,
 4,
 4,
 7,
 4,
 4,
 2,
 7,
 4,
 5,
 9,
 4,
 4,
 9,
 4,
 4,
 4,
 4,
 15,
 4,
 3,
 5,
 7,
 7,
 4,
 7,
 4,
 7,
 4,
 9,
 8,
 7,
 4,
 4,
 4,
 4,
 4,
 14,
 4,
 4,
 4,
 4,
 4,
 10,
 3,
 4,
 9,
 4,
 11,
 4,
 5,
 2,
 4,
 5,
 9,
 5,
 4,
 12,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 7,
 3,
 

[4,
 12,
 21,
 7,
 4,
 4,
 21,
 5,
 21,
 4,
 4,
 7,
 4,
 5,
 9,
 9,
 4,
 7,
 5,
 5,
 4,
 3,
 7,
 7,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 2,
 5,
 7,
 7,
 10,
 4,
 7,
 23,
 5,
 14,
 3,
 7,
 4,
 10,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 2,
 12,
 21,
 4,
 16,
 7,
 4,
 10,
 7,
 7,
 3,
 5,
 10,
 4,
 7,
 4,
 7,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 9,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 11,
 4,
 4,
 4,
 4,
 4,
 7,
 4,
 4,
 4,
 9,
 7,
 4,
 4,
 4,
 4,
 1,
 8,
 4,
 4,
 4,
 5,
 4,
 4,
 10,
 4,
 21,
 9,
 4,
 4,
 21,
 4,
 4,
 4,
 4,
 3,
 4,
 9,
 4,
 5,
 12,
 11,
 7,
 4,
 21,
 4,
 4,
 4,
 9,
 4,
 9,
 4,
 7,
 4,
 21,
 7,
 4,
 4,
 21,
 4,
 14,
 4,
 4,
 4,
 4,
 5,
 9,
 4,
 10,
 12,
 4,
 8,
 4,
 4,
 5,
 4,
 4,
 4,
 5,
 2,
 4,
 4,
 7,
 5,
 10,
 7,
 4,
 4,
 4,
 7,
 4,
 4,
 2,
 7,
 4,
 5,
 9,
 4,
 4,
 9,
 4,
 4,
 4,
 4,
 15,
 4,
 3,
 5,
 7,
 7,
 4,
 7,
 4,
 7,
 4,
 9,
 8,
 7,
 4,
 4,
 4,
 4,
 4,
 14,
 4,
 4,
 4,
 4,
 4,
 10,
 3,
 4,
 9,
 4,
 11,
 4,
 5,
 2,
 4,
 5,
 9,
 5,
 4,
 12,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 7,
 3,
 

In [63]:
pd.Series(final_preds).to_csv('tabnet_final.csv',index=False)